In [ ]:
## ok
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [ ]:
## ok
def make_OHE(orig_df,df_column_name):
    ohe_df = pd.get_dummies(orig_df[df_column_name].astype(str))
    return ohe_df

def standardize_OHE(orig_df, df_column_names):
    row_wise_sum = (orig_df.loc[:,df_column_names]).sum(axis=1)
    orig_df.loc[:,df_column_names] = (orig_df.loc[:,df_column_names]).div(row_wise_sum, axis=0)
    return orig_df.loc[:,df_column_names]

In [ ]:
## ok
FOLDER_PATH = os.getcwd()
FILE_NAME = "\\gdb_join.txt"
FILE_PATH = FOLDER_PATH + "\\" + FILE_NAME
gdb_join_orig = pd.read_csv(FILE_PATH)

FILE_NAME = "\\StreetSegment_LandUse_Subsets.txt"
FILE_PATH = FOLDER_PATH + "\\" + FILE_NAME
StreetSegment_LandUse_Subsets_orig = pd.read_csv(FILE_PATH)

FILE_NAME = "\\Subway_Distances.txt"
FILE_PATH = FOLDER_PATH + "\\" + FILE_NAME
Subway_Distances_orig = pd.read_csv(FILE_PATH)

FILE_NAME = "\\StreetSegment_LandUse_Subsets_dec10.txt"
FILE_PATH = FOLDER_PATH + "\\" + FILE_NAME
StreetSegment_LandUse_Subsets_dec10_orig = pd.read_csv(FILE_PATH)

(StreetSegment_LandUse_Subsets_dec10_orig["LandUse"] == StreetSegment_LandUse_Subsets_orig["LandUse"]).all()

In [ ]:
## ok
gdb_join = gdb_join_orig.copy()
StreetSegment_LandUse_Subsets = StreetSegment_LandUse_Subsets_orig.copy()
Subway_Distances = Subway_Distances_orig.copy()

By now, there should be 2 new tables created after the ArcGIS part:

* **gdb_join** is a table for the street segments, which each street segment having many (usually 8) rows describing its traffic volume at a given 3-hour subdivision of the day
* **StreetSegment_LandUse_Subsets** is a table for the street segments, which each street segment having a variable number of rows that each represent a building or lot within the 500-foot radius of the street segment. We will decide to collapse StreetSegment_LandUse_Subsets so that each street segment has 1 row, essentially aggregating all the info about the street segment's nearby surroundings into 1 compact row.

Both tables have more than 100 columns, so there's a lot of information we don't really need. This next part is about the data columns and selection of which columns are important, and the explained thought process behind their selection.

In [ ]:
## ok

print(len(pd.unique(gdb_join["Segment_ID"])))
print(len(pd.unique(StreetSegment_LandUse_Subsets["lion_Segment_ID"])))
# In the previous nyc_traffic_3HrInterval table, we had 1586 unique street segments. But now, there are only 1463.
# 123 street segments were removed, but which ones?

In [ ]:
## ok

print(len(pd.unique(StreetSegment_LandUse_Subsets["ORIG_FID"])))
print(len(pd.unique(Subway_Distances["IN_FID"])))

### Data Cleaning StreetSegment_LandUse_Subsets
We'll just data clean the table for StreetSegment_LandUse_Subsets. It has a lot of unneeded columns.

Column Transformation Outline
* ['ORIG_FID'] => ['ORIG_FID']
* F(['lion_Segment_ID']) => ['Segment_ID']
* F(['lion_StreetWidt']) => ['StreetWidth']
* F(['lion_SegCount']) => ['SegCount']
* ['NumFloors'] => ['NumFloors']
* F(['RW_TYPE']) => ['RW_TYPE']
* F(['BoroCode']) => ['BoroCode1', 'BoroCode2', 'BoroCode3', 'BoroCode4', 'BoroCode5']
* F([LandUse]) => [LandUse_t* for * in range(?)]
* F([NEAR_DIST]) => ['SubwayProximity'] -Note that this will come from another table

In [ ]:
## ok

SELECTED_SL_COLUMNS_NUMERICAL = []
SELECTED_SL_COLUMNS_CATEGORICAL = []
SELECTED_SL_COLUMNS_OTHERS = []

SELECTED_SL_COLUMNS_NUMERICAL += ['lion_StreetWidt']
SELECTED_SL_COLUMNS_NUMERICAL += ['NumFloors']
SELECTED_SL_COLUMNS_NUMERICAL += ['lion_SegCount']
SELECTED_SL_COLUMNS_NUMERICAL += ['UnitsRes', 'UnitsTotal']


SELECTED_SL_COLUMNS_CATEGORICAL += ['BoroCode']
SELECTED_SL_COLUMNS_CATEGORICAL += ['LandUse']
SELECTED_SL_COLUMNS_CATEGORICAL += ['lion_RW_TYPE']


SELECTED_SL_COLUMNS_OTHERS += ['ORIG_FID']
SELECTED_SL_COLUMNS_OTHERS += ['lion_Segment_ID']

SELECTED_SL_COLUMNS = (SELECTED_SL_COLUMNS_NUMERICAL + 
                       SELECTED_SL_COLUMNS_CATEGORICAL +
                       SELECTED_SL_COLUMNS_OTHERS)

StreetSegment_LandUse_Subsets = StreetSegment_LandUse_Subsets[SELECTED_SL_COLUMNS]

#### Finding Nan's and Imputation

In [ ]:
## ok

StreetSegment_LandUse_Subsets.isnull().any().any()
(StreetSegment_LandUse_Subsets == "").any().any()
(StreetSegment_LandUse_Subsets == " ").any()

In [ ]:
## ok

StreetSegment_LandUse_Subsets = StreetSegment_LandUse_Subsets.replace(" ", np.nan)
StreetSegment_LandUse_Subsets = StreetSegment_LandUse_Subsets.replace("", np.nan)

StreetSegment_LandUse_Subsets.isnull().mean()

In [ ]:
## ok

# Too small to impute, we can discard away. Or IS it?

#StreetSegment_LandUse_Subsets=StreetSegment_LandUse_Subsets.dropna(subset=["LandUse"])
StreetSegment_LandUse_Subsets=StreetSegment_LandUse_Subsets.dropna()
# StreetSegment_LandUse_Subsets=StreetSegment_LandUse_Subsets.fillna(1)

StreetSegment_LandUse_Subsets.isnull().mean()

##### Joining with Subway_Distances

In [ ]:
#assert (len(pd.unique(StreetSegment_LandUse_Subsets["ORIG_FID"]))) == (len(pd.unique(Subway_Distances["IN_FID"])))
## ok
Subway_Distances["SubwayProximity"] = 1/Subway_Distances["NEAR_DIST"]

StreetSegment_Subway_Distances = Subway_Distances.groupby(["IN_FID"], as_index=False)["SubwayProximity"].sum()

StreetSegment_Subway_Distances = StreetSegment_Subway_Distances.rename(columns={"IN_FID": "ORIG_FID"})

StreetSegment_LandUse_Subsets = StreetSegment_LandUse_Subsets.merge(StreetSegment_Subway_Distances, on="ORIG_FID")

#### Column Transformations

In [ ]:
StreetSegment_LandUse_Subsets["LandUse"].isnull().any()

OVERLAP_THRESHOLD = 0.5

LANDUSE_COLUMN_VALUES = []

LANDUSE_COLUMN_VALUES += [1*(
    StreetSegment_LandUse_Subsets["LandUse"].isin(["01", "02", "03"])
 + OVERLAP_THRESHOLD*(StreetSegment_LandUse_Subsets["LandUse"].isin(["04"])))]

LANDUSE_COLUMN_VALUES += [1*(
    StreetSegment_LandUse_Subsets["LandUse"].isin(["05"])
 + OVERLAP_THRESHOLD*(StreetSegment_LandUse_Subsets["LandUse"].isin(["04"])))]

LANDUSE_COLUMN_VALUES += [1*(StreetSegment_LandUse_Subsets["LandUse"].isin(["06", "07", "10"]))]

LANDUSE_COLUMN_VALUES += [1*(StreetSegment_LandUse_Subsets["LandUse"].isin(["08"]))]
## ok
LANDUSE_COLUMN_VALUES += [1*(StreetSegment_LandUse_Subsets["LandUse"].isin(["09", "11"]))]

## ok
LANDUSE_COLUMNS = ["LandUse_t{0}".format(i+1) for i in range(len(LANDUSE_COLUMN_VALUES))]

## ok
for i in range(len(LANDUSE_COLUMNS)):
    column_name = LANDUSE_COLUMNS[i]
    column = LANDUSE_COLUMN_VALUES[i]
    StreetSegment_LandUse_Subsets[column_name] = column 

In [ ]:
## ok

StreetSegment_LandUse_Subsets["BoroCode"] = "BoroCode" + StreetSegment_LandUse_Subsets["BoroCode"].astype(str)
SSLU_Borough_OHE = make_OHE(StreetSegment_LandUse_Subsets, "BoroCode")

In [ ]:
## Exclusive
SSLU_LandUse = StreetSegment_LandUse_Subsets.loc[:,LANDUSE_COLUMNS]

In [ ]:
## ok

StreetSegment_LandUse_Subsets["StreetWidth"] = StreetSegment_LandUse_Subsets["lion_StreetWidt"]

In [ ]:
## ok

StreetSegment_LandUse_Subsets["StreetWidth_z"] = (
    (StreetSegment_LandUse_Subsets["StreetWidth"] - StreetSegment_LandUse_Subsets["StreetWidth"].mean())/
    StreetSegment_LandUse_Subsets["StreetWidth"].std()
)

In [ ]:
## ok

StreetSegment_LandUse_Subsets["Segment_ID"] = StreetSegment_LandUse_Subsets["lion_Segment_ID"]

In [ ]:
## exclusive
StreetSegment_LandUse_Subsets["SegCount"] = StreetSegment_LandUse_Subsets["lion_SegCount"]

In [ ]:
## Exclusive
StreetSegment_LandUse_Subsets["NumFloors"] = StreetSegment_LandUse_Subsets["NumFloors"]

In [ ]:
## Exclusive
StreetSegment_LandUse_Subsets["RW_TYPE"] = StreetSegment_LandUse_Subsets["lion_RW_TYPE"]
StreetSegment_LandUse_Subsets["RW_TYPE"] = StreetSegment_LandUse_Subsets["RW_TYPE"].astype(str).str.lstrip().astype(int)
SSLU_RW_TYPE = StreetSegment_LandUse_Subsets[["RW_TYPE"]]

In [ ]:
## Exclusive

StreetSegment_LandUse_Subsets["UnitsRes_Prop"] = (
    StreetSegment_LandUse_Subsets["UnitsRes"]/StreetSegment_LandUse_Subsets["UnitsTotal"]
)

##### Final Aggregation Preparation 

In [ ]:
assert (SSLU_LandUse==StreetSegment_LandUse_Subsets.loc[:,LANDUSE_COLUMNS]).all().all()

In [ ]:
## ok

Concat_inputs = [
    StreetSegment_LandUse_Subsets["StreetWidth"],
    StreetSegment_LandUse_Subsets["StreetWidth_z"],
    StreetSegment_LandUse_Subsets["NumFloors"],
    StreetSegment_LandUse_Subsets["SegCount"],
    StreetSegment_LandUse_Subsets["UnitsRes_Prop"],
    StreetSegment_LandUse_Subsets["SubwayProximity"],
    SSLU_LandUse,
    SSLU_Borough_OHE,
    SSLU_RW_TYPE,
    StreetSegment_LandUse_Subsets["ORIG_FID"],
    StreetSegment_LandUse_Subsets["Segment_ID"]
]

StreetSegment_LandUse_Subsets_Grouping = pd.concat(Concat_inputs, axis=1)

##### Final Aggregation

In [ ]:
## ok


StreetSegment_LandUse_Subsets_Aggregation = dict()

StreetSegment_LandUse_Subsets_Aggregation.update(
    {
    "NumFloors": np.mean,
    "SegCount": np.mean, # will be the same throughout
    "RW_TYPE": lambda x: pd.Series(x).mode()[0],
    "StreetWidth": np.mean,
    "StreetWidth_z": np.mean,
    "UnitsRes_Prop": np.mean,
    "SubwayProximity": np.mean,
    }
)

StreetSegment_LandUse_Subsets_Aggregation.update(
    dict([(col, np.sum) for col in SSLU_Borough_OHE.columns.values])
)

StreetSegment_LandUse_Subsets_Aggregation.update(
    dict([(col, np.sum) for col in SSLU_LandUse.columns.values])
)

In [ ]:
## ok

Unique_StreetSegment_LandUse_Subsets = StreetSegment_LandUse_Subsets_Grouping.groupby(["Segment_ID"], as_index=False).agg(
    StreetSegment_LandUse_Subsets_Aggregation
)

In [ ]:
## ok


OHE_columns = []

OHE_columns += list(SSLU_Borough_OHE.columns.values)
OHE_columns += LANDUSE_COLUMNS

non_OHE_columns = (
    set(Unique_StreetSegment_LandUse_Subsets.columns.values) - set(OHE_columns))

In [ ]:
## ok

Concat_inputs = [
    Unique_StreetSegment_LandUse_Subsets[list(non_OHE_columns)],
    standardize_OHE(Unique_StreetSegment_LandUse_Subsets, list(SSLU_Borough_OHE.columns.values)),
    standardize_OHE(Unique_StreetSegment_LandUse_Subsets, LANDUSE_COLUMNS) # added 2021/12/10
]

Unique_StreetSegment_LandUse_Subsets = pd.concat(Concat_inputs, axis=1)

In [ ]:
## ok

Unique_StreetSegment_LandUse_Subsets.isin([np.inf, -np.inf]).any().any()
Unique_StreetSegment_LandUse_Subsets = Unique_StreetSegment_LandUse_Subsets.fillna(0)

In [ ]:
for i in range(1, 6, 1):
    colname = "LandUse_t{0}".format(i)
    print(colname, Unique_StreetSegment_LandUse_Subsets[colname].sum())

In [ ]:
plt.hist(Unique_StreetSegment_LandUse_Subsets["LandUse_t1"], 50)

In [ ]:
plt.hist(Unique_StreetSegment_LandUse_Subsets["UnitsRes_Prop"], 50)

##### Reformatting for EDA

In [ ]:

## ok

Unique_StreetSegment_LandUse_Subsets_EDA = Unique_StreetSegment_LandUse_Subsets.copy()

BOROCODE_NAMES = ["Manhattan", "Bronx", "Brooklyn", "Queens", "Staten Island"]

Unique_StreetSegment_LandUse_Subsets_EDA["BoroCode"] = np.argmax(Unique_StreetSegment_LandUse_Subsets_EDA[["BoroCode1", "BoroCode2", "BoroCode3", "BoroCode4", "BoroCode5"]].to_numpy(), axis=1)+1
Unique_StreetSegment_LandUse_Subsets_EDA = Unique_StreetSegment_LandUse_Subsets_EDA.drop(
    columns = ["BoroCode{0}".format(i+1) for i in range(len(pd.unique(Unique_StreetSegment_LandUse_Subsets_EDA["BoroCode"])))]
                                                                                        )

Unique_StreetSegment_LandUse_Subsets_EDA["BoroCode"] = Unique_StreetSegment_LandUse_Subsets_EDA["BoroCode"].apply(
    lambda x: BOROCODE_NAMES[x - 1])

In [ ]:
RW_TYPE_NAMES = [
    "Street", "Highway", "Bridge", "Tunnel", "Boardwalk", "Path/Trail", "Step Street", 
    "Driveway", "Ramp", "Alley", "Unknown", "Non-Physical Street Segment", "U-Turn", "Ferry Route"
]


Unique_StreetSegment_LandUse_Subsets_EDA["RW_TYPE"] = Unique_StreetSegment_LandUse_Subsets_EDA["RW_TYPE"].astype(int).apply(lambda x: RW_TYPE_NAMES[x-1])

In [ ]:
FILE_NAME = "\\Unique_StreetSegment_LandUse_Subsets_EDA.csv"
FILE_PATH = FOLDER_PATH + FILE_NAME
Unique_StreetSegment_LandUse_Subsets_EDA.to_csv(FILE_PATH, index=False)

##### Reformatting for Modelling

In [ ]:
Unique_StreetSegment_LandUse_Subsets_ml = Unique_StreetSegment_LandUse_Subsets.copy()

In [ ]:
#Unique_StreetSegment_LandUse_Subsets_ml["RW_TYPE"] = "RW_"+(Unique_StreetSegment_LandUse_Subsets_ml["RW_TYPE"].astype(int).astype(str))
#SSLU_RW_TYPE_OHE = make_OHE(Unique_StreetSegment_LandUse_Subsets_ml, "RW_TYPE")
Unique_StreetSegment_LandUse_Subsets_ml = Unique_StreetSegment_LandUse_Subsets_ml.drop(columns=["RW_TYPE"])

Unique_StreetSegment_LandUse_Subsets_ml = Unique_StreetSegment_LandUse_Subsets_ml.drop(columns=["StreetWidth"])
Unique_StreetSegment_LandUse_Subsets_ml = Unique_StreetSegment_LandUse_Subsets_ml.rename(
    columns={"StreetWidth_z": "StreetWidth"})

In [ ]:
Concat_inputs = [
    #SSLU_RW_TYPE_OHE,
    Unique_StreetSegment_LandUse_Subsets_ml,
]
Unique_StreetSegment_LandUse_Subsets_ml = pd.concat(Concat_inputs, axis=1)

In [ ]:
FILE_NAME = "\\Unique_StreetSegment_LandUse_Subsets_ml.csv"
FILE_PATH = FOLDER_PATH + FILE_NAME
Unique_StreetSegment_LandUse_Subsets_ml.to_csv(FILE_PATH, index=False)

### Data Cleaning gdb_join

#### Column Transformation
* F(['Is_Weekend']) => ['Is_Weekend']
* F(['Traffic_Volume']) => ['Traffic_Volume']
* F(['Season']) => ['Season1', 'Season2', 'Season4']
* F(['Segment_ID']) => ['Segment_ID']
* F(['F3_Hr_Intvl']) => ['3_Hr_Intvl']

In [ ]:
SELECTED_GJ_COLUMNS_NUMERICAL = []
SELECTED_GJ_COLUMNS_CATEGORICAL = []
SELECTED_GJ_COLUMNS_OTHERS = []

SELECTED_GJ_COLUMNS_NUMERICAL += ['Traffic_Volume']

SELECTED_GJ_COLUMNS_CATEGORICAL += ['Is_Weekend']
SELECTED_GJ_COLUMNS_CATEGORICAL += ['F3_Hr_Intvl']
SELECTED_GJ_COLUMNS_CATEGORICAL += ['Season']

SELECTED_GJ_COLUMNS_OTHERS += ['Segment_ID']

SELECTED_GJ_COLUMNS = (SELECTED_GJ_COLUMNS_NUMERICAL + 
                       SELECTED_GJ_COLUMNS_CATEGORICAL +
                       SELECTED_GJ_COLUMNS_OTHERS)

gdb_join = gdb_join[SELECTED_GJ_COLUMNS]

##### Reformatting for EDA

In [ ]:
gdb_join_EDA = gdb_join.copy()

In [ ]:
gdb_join_EDA["Is_Weekend"] = gdb_join["Is_Weekend"]
gdb_join_EDA["Season"] = gdb_join["Season"]
gdb_join_EDA["Segment_ID"] = gdb_join["Segment_ID"]
gdb_join_EDA["3_Hr_Intvl"] = gdb_join["F3_Hr_Intvl"]
gdb_join_EDA["Traffic_Volume"] = gdb_join["Traffic_Volume"] 

SEASON_NAMES = ["Winter", "Spring", "Summer", "Autumn"]

gdb_join_EDA["Season"] = gdb_join_EDA["Season"].apply(
    lambda x: SEASON_NAMES[x-1])

In [ ]:
Concat_inputs = [
    gdb_join_EDA[["Segment_ID"]],
    gdb_join_EDA[["Is_Weekend"]],
    gdb_join_EDA[["3_Hr_Intvl"]],
    gdb_join_EDA[["Season"]],
    gdb_join_EDA[["Traffic_Volume"]],
]
gdb_join_EDA = pd.concat(Concat_inputs, axis=1)

In [ ]:
FILE_NAME = "\\gdb_join_EDA.csv"
FILE_PATH = FOLDER_PATH + FILE_NAME
gdb_join_EDA.to_csv(FILE_PATH, index=False)

##### Reformatting for Modelling

In [ ]:
gdb_join_ml = gdb_join.copy()

In [ ]:
def day_night_transform(x):
    # flattens the range 0 to 24 to be between 0 and 1 as a bellcurve
    return 2 * (1 + np.exp(((x - 12)/4)**2))**(-1)

gdb_join_ml["Is_Weekend"] = gdb_join["Is_Weekend"].astype(int)

gdb_join_ml["Season"] = "Season"+gdb_join["Season"].astype(int).astype(str)
gj_season_OHE = make_OHE(gdb_join_ml, "Season")

gdb_join_ml["Daylight"] = gdb_join["F3_Hr_Intvl"].apply(day_night_transform)

gdb_join_ml["F3_Hr_Intvl"] = "Hr"+gdb_join["F3_Hr_Intvl"].astype(int).astype(str)
gj_hour_OHE = make_OHE(gdb_join_ml, "F3_Hr_Intvl")
gj_hour_OHE = gj_hour_OHE[["Hr"+str(i) for i in range(3,27,3)]]

In [ ]:
Concat_inputs = [
    gdb_join_ml[["Segment_ID"]],
    gdb_join_ml[["Is_Weekend"]],
    gdb_join_ml[["Daylight"]],
    gj_season_OHE,
    gj_hour_OHE,
    gdb_join_ml[["Traffic_Volume"]],
]

gdb_join_ml = pd.concat(Concat_inputs, axis=1)

In [ ]:
FILE_NAME = "\\gdb_join_ml.csv"
FILE_PATH = FOLDER_PATH + FILE_NAME
gdb_join_ml.to_csv(FILE_PATH, index=False)

In [ ]:
#|

In [ ]:
FOLDER_PATH = os.getcwd()
FILE_NAME = "\\gdb_join_EDA.csv"
FILE_PATH = FOLDER_PATH + "\\" + FILE_NAME
gdb_join_EDA_orig = pd.read_csv(FILE_PATH)

FILE_NAME = "\\Unique_StreetSegment_LandUse_Subsets_EDA.csv"
FILE_PATH = FOLDER_PATH + "\\" + FILE_NAME
Unique_StreetSegment_LandUse_Subsets_EDA_orig = pd.read_csv(FILE_PATH)

gdb_join_EDA = gdb_join_EDA_orig.copy()
Unique_StreetSegment_LandUse_Subsets_EDA = Unique_StreetSegment_LandUse_Subsets_EDA_orig.copy()

JOINER_COLUMN = ["Segment_ID"]
nyc_traffic_EDA_orig = gdb_join_EDA.merge(Unique_StreetSegment_LandUse_Subsets_EDA, on=JOINER_COLUMN)

FILE_NAME = "\\nyc_traffic_EDA_orig.csv"
FILE_PATH = FOLDER_PATH + FILE_NAME
nyc_traffic_EDA_orig.to_csv(FILE_PATH, index=False)

In [ ]:
FOLDER_PATH = os.getcwd()
FILE_NAME = "\\gdb_join_ml.csv"
FILE_PATH = FOLDER_PATH + "\\" + FILE_NAME
gdb_join_ml_orig = pd.read_csv(FILE_PATH)

FILE_NAME = "\\Unique_StreetSegment_LandUse_Subsets_ml.csv"
FILE_PATH = FOLDER_PATH + "\\" + FILE_NAME
Unique_StreetSegment_LandUse_Subsets_ml_orig = pd.read_csv(FILE_PATH)

gdb_join_ml = gdb_join_ml_orig.copy()
Unique_StreetSegment_LandUse_Subsets_ml = Unique_StreetSegment_LandUse_Subsets_ml_orig.copy()

JOINER_COLUMN = ["Segment_ID"]
nyc_traffic_ml_orig = gdb_join_ml.merge(Unique_StreetSegment_LandUse_Subsets_ml, on=JOINER_COLUMN)

FILE_NAME = "\\nyc_traffic_ml_orig.csv"
FILE_PATH = FOLDER_PATH + FILE_NAME
nyc_traffic_ml_orig.to_csv(FILE_PATH, index=False)

In [ ]:
#/